In [ ]:
from OperacionesCSV import importar_CSV, consultar_atributos
from Neo4HungerGames import Neo4HungerGames
import pandas as pd
from flask import Flask, render_template, redirect, url_for, session, request
import os
import uuid
from Controller import Controller

ruta_csv='HungerGames_Characters_Dataset_ALL.csv'

In [24]:
hgDF = importar_CSV(ruta_csv)
hgDF['ID'] = hgDF.index

uri = "bolt://localhost:7687"
user = "mdad"
password = "123456789"

neoConnection = Neo4HungerGames(uri, user, password)
neoConnection.create_nodes_links_from_df(hgDF)

--------------------------------------------
Se ha importado el archivo CSV correctamente


In [22]:
app = Flask(__name__)

# Ruta principal
@app.route('/')
def home():
    return render_template('index.html')


@app.route('/characters', methods=['GET'])
def characters():
    return render_template('characters.html')

@app.route('/characters/all', methods=['GET'])
def all_characters():
    characters = neoConnection.get_all_characters()
    return render_template('characters_list.html', characters=characters)

@app.route('/characters/create', methods=['POST'])
def create_character():
    return Controller.create_character(neoConnection, request=request)


In [ ]:
if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=8080,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://158.49.188.38:8080
Press CTRL+C to quit
127.0.0.1 - - [12/May/2025 18:13:33] "GET /characters HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 18:13:37] "GET /characters/all HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 18:13:43] "GET /characters/all HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 18:13:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 18:13:48] "GET /characters HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 18:13:49] "GET /characters/all HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 18:13:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 18:13:54] "GET /characters HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 18:13:56] "GET /characters/272 HTTP/1.1" 404 -
127.0.0.1 - - [12/May/2025 18:14:08] "GET /characters/all HTTP/1.1" 200 -
